# Aliqouting Protocol

This protocol aliquots a protein in a tube rack (A1) to a user specified number of PCR tubes. 

In [19]:
# !pip install ipywidgets
from opentrons import simulate, execute
from opentrons.types import Location, Point
# import ipywidgets as widgets

import numpy as np
import pandas as pd
import time

Load the protocol and home the robot:

In [20]:
protocol = simulate.get_protocol_api('2.11')

protocol.home()

C:\Users\raras\.opentrons\robot_settings.json not found. Loading defaults
C:\Users\raras\.opentrons\deck_calibration.json not found. Loading defaults


### Load the labware with the following setup:

<table style="width:300px;">
    <tr><td>10</td><td>11</td><td>trash</td></tr>
    <tr><td>7</td><td>8</td><td>9</td></tr>
    <tr><td>4</td><td>5</td><td>6</td></tr>
    <tr><td>1 <br> tips 300</td><td>2 <br> Aluminium block</td><td>3 <br> tube rack 1 (protein) </td></tr>
    
</table>

In [21]:
tips_300 = [protocol.load_labware("opentrons_96_tiprack_300ul", 1)]

# tips_20 = [protocol.load_labware("opentrons_96_filtertiprack_20ul", 4)]

pcr_tubes_plate = protocol.load_labware("opentrons_96_aluminumblock_generic_pcr_strip_200ul", 2)

tube_rack1 = protocol.load_labware("opentrons_10_tuberack_nest_4x50ml_6x15ml_conical", 3)



### Load pipette

In [22]:
pip = protocol.load_instrument("p300_single_gen2", "left", tip_racks=tips_300)

In [23]:
pip.starting_tip = tips_300[0].well("B2")

### Load fluids

In [24]:
protein = {}
protein["pos"] = tube_rack1["A1"]
protein["v_t"] = float(input("What is the transfer volume in µl?"))
protein["v"] = float(input("What is the total volume in µl?"))

eppis = {}
eppis["n"] = int(input("How many PCR tubes do you want to fill"))
eppis["wells"] = pcr_tubes_plate.wells()[0:eppis["n"]]

if protein["v_t"]*eppis["n"] > protein["v"]:
    raise ValueError( f'Not enough protein to fill { eppis["n"] } tubes with {protein["v_t"]} µl each' )

What is the transfer volume in µl? 50
What is the total volume in µl? 5000
How many PCR tubes do you want to fill 10


### Start Pipetting

The pipette will aspirate 20 µl of protein and distribute it to the specified number of PCR tubes. Please fill the PCR tubes starting from top to bottom in the first column, then from top to bottom the remaining columns. 

In [46]:
if not pip.hw_pipette["has_tip"]:
    pip.pick_up_tip()

wells = pcr_tubes_plate.wells()[0:eppis["n"]]

for well in wells:
    
    # primitive height tracking. 1 ml corresponds to ~ .66 mm
    if protein["v"] > 2000:
        z_offset = (2/3 * (protein["v"]- 2500)  / 100) + 25
        pip.well_bottom_clearance.aspirate = z_offset
    else:
        pip.well_bottom_clearance.aspirate = 1.0
       
    print(f'Aspirating { protein["v_t"]:.2f} µl protein at {z_offset:.2f} mm above bottom of the well')

    pip.aspirate(protein["v_t"],
                 protein["pos"],
                )

    print('Dispensing...')
    protein["v"] = protein["v"]-protein["v_t"]
    
    
    pip.dispense(protein["v_t"],
                 well
                )


Aspirating 50.00 µl protein at 24.00 mm above bottom of the well
Dispensing...
Aspirating 50.00 µl protein at 23.67 mm above bottom of the well
Dispensing...
Aspirating 50.00 µl protein at 23.33 mm above bottom of the well
Dispensing...
Aspirating 50.00 µl protein at 23.00 mm above bottom of the well
Dispensing...
Aspirating 50.00 µl protein at 22.67 mm above bottom of the well
Dispensing...
Aspirating 50.00 µl protein at 22.33 mm above bottom of the well
Dispensing...
Aspirating 50.00 µl protein at 22.00 mm above bottom of the well
Dispensing...
Aspirating 50.00 µl protein at 22.00 mm above bottom of the well
Dispensing...
Aspirating 50.00 µl protein at 22.00 mm above bottom of the well
Dispensing...
Aspirating 50.00 µl protein at 22.00 mm above bottom of the well
Dispensing...


In [47]:
pip.drop_tip()
protocol.home()